# Load data

In [ ]:
import glob
import os

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.text import TextLoader
loader = DirectoryLoader("un/TXT/Session 78 - 2023/", glob="**/*.txt", loader_cls=TextLoader)
data = loader.load()

In [ ]:
data[0:5]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
all_splits = text_splitter.split_documents(data)

In [ ]:
all_splits[0:10]

In [ ]:
# this used to work and is now deprecated
# from langchain.embeddings import HuggingFaceEmbeddings
# this will work now and hopefully also in the futur in the future
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.sklearn import SKLearnVectorStore

retriever = SKLearnVectorStore.from_documents(all_splits, HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))\
                              .as_retriever(search_kwargs={"k": 100})

In [ ]:
question = "Is the climate crisis worse for poorer countries?"
docs = retriever.invoke(question)
len(docs)

In [ ]:
docs[0:10]

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)
df = pd.DataFrame([{"source": d.metadata["source"], "text": d.page_content} for d in docs])
df

In [ ]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever


cross_encoder = HuggingFaceCrossEncoder(model_name="mixedbread-ai/mxbai-rerank-large-v1")
compressor = CrossEncoderReranker(model=cross_encoder, top_n=20)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
compressed_docs = compression_retriever.invoke(question)

In [ ]:
pd.DataFrame([{"source": d.metadata["source"], "text": d.page_content} for d in compressed_docs])